In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import time
import matplotlib.pyplot as plt
import random

In [3]:
os.chdir("/content/drive/My Drive/Anomaly detection - BuildSys2020/")

In [4]:
# For reproducibility
np.random.seed(12)
tf.random.set_seed(12)
random.seed(1)

In [5]:
def usable_features(df_building, features_used):  # for droping the features not to be considered in the model
  ## df_building - pandas dataframe of the datset
  ## features_used - features to be included in the cleaned dataset

  # creating list of columns that are not used 
  col = df_building.columns
  features_not_used = [] # store columns that won't be used
  for x in col:
    if x not in features_used:
      features_not_used.append(x)

  # drop the columns
  df_building= df_building.drop(features_not_used, axis = 1) # dropping not used features 

  # to fill NA data we simply replace the values with 0
  df_building = df_building.fillna(0)
  
  return df_building

In [6]:
def normalize_minmax(df, col):  # function to normalize the columns 
    ## df - data frame containing the columns
    ## columns to be normalized  
    result = df.copy()
    for feature_name in col:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()

        print("Feature Name : {}  :  Max Value - {} ; Min Value - {}".format(feature_name, max_value, min_value))
        # normalize 
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result 

In [7]:
def normalize_zAlgo(df, col):  # function to normalize the columns 
    ## df - data frame containing the columns
    ## columns to be normalized  
    result = df.copy()
    for feature_name in col:
        std_value = df[feature_name].std()
        mean_value = df[feature_name].mean()

        print("Feature Name : {}  :  Std Value - {} ; Mean Value - {}".format(feature_name, std_value, mean_value))
        # normalize 
        result[feature_name] = (df[feature_name] - mean_value) / std_value
    return result

In [8]:
def clean_dataset_ashrae(dataset, meter_types, features_used, z_columns, minmax_columns):
  ## meter_types - list of meters to be included 
  ## features_used - list of features to be included in the final data
  ## z_columns - list of columns to be normalized with Z algorithm
  ## minmax_columns - list of columns to be normalized with min-max normalization

  meter_dataset = dataset.loc[dataset['meter']==0]
  
  # create a new column  
  meter_dataset['meter_reading_log'] = np.log(meter_dataset['meter_reading']+1) 

  #convert categorical data type to int
  meter_dataset['primary_use'] = meter_dataset['primary_use'].astype('category')
  unique_primUse_list = list(meter_dataset.primary_use.unique())
  unique_primUse_dict = {unique_primUse_list[i]: i for i in range(0, len(unique_primUse_list))}
  meter_dataset['primary_use'] = meter_dataset['primary_use'].map(unique_primUse_dict).astype(int)

  # remove columns that won't be included in the model's input 
  df_train = usable_features(meter_dataset, features_used)

  # normalize features 
  df_train_norm1 = normalize_zAlgo(df_train, z_columns) # z-algorithm
  df_train_norm = normalize_zAlgo(df_train_norm1, minmax_columns) # min-max scaler

  print("df_train_norm shape {}".format(df_train_norm.shape))
  return df_train_norm

In [9]:
## creating input sequence of length - seq_length
def create_windows(arr_data, arr_anomaly_label, arr_op, seq_length, nb_features):
  ## arr_data - normalized dataset as a numpy array 
  ## arr_op - array containing anomaly labels for each time step 
  ## seq_length - wiindow length 
  ## nb_features - no. of features to be used as input 

  anom_x = [] # storing anomalous windows - input 
  non_anom_x = [] # storing non-anomalous windows - input 
  anom_y = [] # storing anomalous windows - output 
  non_anom_y = [] # storing non-anomalous windows - output 

  # run a loop to move a seq_length size window across data non-overlapping
  for i in range(0,arr_data.shape[0]//seq_length):

    # slice the window 
    window_features = arr_data[i*seq_length:(i+1)*seq_length].reshape((seq_length, nb_features))   # window of seq_length
    window_output = arr_op[i*seq_length:(i+1)*seq_length].reshape((seq_length,1))

    cond_input = np.concatenate((window_output, window_features[0][0].reshape((1,1))))

    is_anomaly = np.count_nonzero(arr_anomaly_label[i*seq_length:(i+1)*seq_length])  #if even at one time point anomaly is present the window would be considered anomalous
    # print(is_anomaly)

    if is_anomaly > 0 :  # separating the anomalous and non-anomalous data 
      anom_x.append(cond_input)
      anom_y.append(window_output)
    else:
      non_anom_x.append(cond_input)
      non_anom_y.append(window_output)

  return non_anom_x, non_anom_y, anom_x, anom_y 

In [10]:
def mix_data(data_x, data_y):
  #  Mix Data (to make it similar to i.i.d)
  data_idx = np.random.permutation(len(data_x))


  output_data_x = []  # Store shuffled data
  output_data_y = []

  for i in range(len(data_x)):
    output_data_x.append(data_x[data_idx[i]])
    output_data_y.append(data_y[data_idx[i]])

  print("ouput_x shape {} , {}".format(len(output_data_x), output_data_x[0].shape))
  return output_data_x, output_data_y

In [11]:
with open('ashrae_rank1_train_clean.pkl', 'rb') as f:
      dataset = pickle.load(f)

In [12]:
def load_data_ashrae(dataset, meter_types, features_used, z_columns, minmax_columns, seq_length, nb_buildings, output_feature):
  ## seq_length - window length 
  ## nb_buildings - no of buildings for the input samples 
  ## output_feature - feature that would be taken as the output to be predicted 

  ## import the dataset 
  

  df_final_train_norm = clean_dataset_ashrae(dataset, meter_types, features_used, z_columns, minmax_columns)

  # lists to store final input data  
  anom_x = []
  anom_y = []
  non_anom_x = []
  non_anom_y = []

  # group data on the basis of their building_id 
  grp_data = df_final_train_norm.groupby('building_id')

  # create a random list of building whose data would be considered 
  list_buildings = random.sample(list(grp_data.groups.keys()), nb_buildings)

  # loop throught the building ids - segregate them into Anomalous and Non-Anomalouws windows
  for grp_no in list_buildings:

    # pick a building id whose data needs to be added into final input data
    building_data = grp_data.get_group(grp_no)
    #building_id is not needed anymore 
    building_data = building_data.drop('building_id', axis = 1) # dropping not used features 

    # separating labels from the data 
    arr_labels = building_data['is_bad_meter_reading'].to_numpy()
    # label is not not need anymore 
    building_data = building_data.drop('is_bad_meter_reading', axis = 1) # dropping not used features 

    # separating output_feature from the data 
    arr_output = building_data[output_feature].to_numpy()
    # output_feature is not not need anymore 
    building_data = building_data.drop(output_feature, axis = 1) # dropping not used features 

    # making sure the data is sequential 
    building_data.sort_values("timestamp", axis = 0, ascending = True) 
    building_data = building_data.drop('timestamp', axis = 1) # dropping not used features 

    # creating numpy array of remaining features 
    building_data = building_data.to_numpy()

    # creating windowed data
    nb_features = building_data.shape[1]
    na_x, na_y, a_x, a_y = create_windows(building_data, arr_labels, arr_output, seq_length, nb_features)

    print(" na_x - len {}, shape {} ".format(len(na_x), na_x[0].shape))
    # print(" a_x - len {}".format(len(a_x)))
    # accumulating single bulding data 
    anom_x.extend(a_x)
    anom_y.extend(a_y)
    non_anom_x.extend(na_x)
    non_anom_y.extend(na_y)

  # making data similar to i.i.d.
  anom_x, anom_y = mix_data(anom_x, anom_y)
  non_anom_x, non_anom_y = mix_data(non_anom_x, non_anom_y)

  return non_anom_x, non_anom_y, anom_x, anom_y 

In [13]:
## change these lists as per model's input 
meter_types = [0] 

features_used = ['building_id','timestamp', 'month', 'is_bad_meter_reading','meter_reading_log']

z_columns = []          

minmax_columns = ['month']

seq_length = 24 


nb_buildings = 145

output_feature = 'meter_reading_log'

In [14]:
non_anom_x, non_anom_y, anom_x, anom_y = load_data_ashrae(dataset, meter_types, features_used, z_columns, minmax_columns, seq_length, nb_buildings, output_feature)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

Feature Name : month  :  Std Value - 3.4435748699115054 ; Mean Value - 6.552057846381409
df_train_norm shape (12060910, 5)
 na_x - len 354, shape (25, 1) 
 na_x - len 351, shape (25, 1) 
 na_x - len 361, shape (25, 1) 
 na_x - len 353, shape (25, 1) 
 na_x - len 357, shape (25, 1) 
 na_x - len 361, shape (25, 1) 
 na_x - len 339, shape (25, 1) 
 na_x - len 342, shape (25, 1) 
 na_x - len 311, shape (25, 1) 
 na_x - len 365, shape (25, 1) 
 na_x - len 363, shape (25, 1) 
 na_x - len 340, shape (25, 1) 
 na_x - len 268, shape (25, 1) 
 na_x - len 225, shape (25, 1) 
 na_x - len 252, shape (25, 1) 
 na_x - len 341, shape (25, 1) 
 na_x - len 335, shape (25, 1) 
 na_x - len 224, shape (25, 1) 
 na_x - len 343, shape (25, 1) 
 na_x - len 339, shape (25, 1) 
 na_x - len 365, shape (25, 1) 
 na_x - len 332, shape (25, 1) 
 na_x - len 339, shape (25, 1) 
 na_x - len 365, shape (25, 1) 
 na_x - len 224, shape (25, 1) 
 na_x - len 178, shape (25, 1) 
 na_x - len 225, shape (25, 1) 
 na_x - len 2

In [15]:
def split_data(train_percent, val_percent, test_percent, output_non_anom_x, output_non_anom_y ):

  # train dataset
  X_train_non_anom = output_non_anom_x[:int((len(output_non_anom_y)*train_percent))]
  Y_train_non_anom = output_non_anom_y[:int((len(output_non_anom_y)*train_percent))]

  # validation dataset
  X_val_non_anom = output_non_anom_x[int((len(output_non_anom_y)*train_percent)):-int((len(output_non_anom_y)*test_percent) )]
  Y_val_non_anom = output_non_anom_y[int((len(output_non_anom_y)*train_percent)):-int((len(output_non_anom_y)*test_percent) )]

  # test dataset                                                          
  X_test_non_anom = output_non_anom_x[-int(len(output_non_anom_y)*test_percent):]
  Y_test_non_anom = output_non_anom_y[-int(len(output_non_anom_y)*test_percent):]     

  return  X_train_non_anom, Y_train_non_anom, X_val_non_anom, Y_val_non_anom, X_test_non_anom, Y_test_non_anom

In [16]:
train_percent = 0.8
val_percent = 0.1
test_percent = 0.1

In [17]:
X_train_non_anom, Y_train_non_anom, X_val_non_anom, Y_val_non_anom, X_test_non_anom, Y_test_non_anom = split_data(train_percent, val_percent, test_percent, non_anom_x, non_anom_y)

In [18]:
from numpy import percentile

In [19]:
X_train_anom, Y_train_anom, X_val_anom, Y_val_anom, X_test_anom, Y_test_anom = split_data(train_percent, val_percent, test_percent, anom_x, anom_y)

In [21]:
os.chdir("/content/drive/My Drive/ASHRAEData/")

In [22]:
## libraries to create the model 
from tensorflow.keras.layers import  MaxPool1D, Input
from tensorflow.keras.layers import  Dense, Activation, Dropout, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K

In [23]:
tf.keras.backend.clear_session()

In [24]:
def compile_model(model):
    opt = Adam(lr=0.0001)
    model.compile(optimizer=opt, loss='mse', metrics='mse')
    return model

In [25]:
def generate_model_autoencoder1(input_len, name):
  input_encoder = Input(shape=(input_len,))  # input layer
  ip = Dropout(0.05)(input_encoder)
  
  encoder = Dense(6, activation='tanh')(ip)

  encoded = Dense(3, activation='tanh')(encoder)  # encoder ouput 

  decoder = Dense(6, activation='tanh')(encoded)

  decoded = Dense(input_len, activation='relu')(decoder) # decoder output 
  
  model = Model(input_encoder, decoded, name = name)

  return model

In [26]:
autoencoder1 = generate_model_autoencoder1(24,  "AutoencoderBasic-")

In [27]:
autoencoder1 = compile_model(autoencoder1)

In [28]:
## create dataset 
train_dataset_autoencoder1 = tf.data.Dataset.from_tensor_slices((Y_train_non_anom,Y_train_non_anom))

val_dataset_autoencoder1 = tf.data.Dataset.from_tensor_slices((Y_val_non_anom, Y_val_non_anom))


BATCH_SIZE_TRAIN = 32
BATCH_SIZE_VAL = 16

train_dataset_autoencoder1 = train_dataset_autoencoder1.batch(BATCH_SIZE_TRAIN)
val_dataset_autoencoder1 = val_dataset_autoencoder1.batch(BATCH_SIZE_VAL)
print(train_dataset_autoencoder1)
print(val_dataset_autoencoder1)

<BatchDataset shapes: ((None, 24, 1), (None, 24, 1)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 24, 1), (None, 24, 1)), types: (tf.float32, tf.float32)>


In [29]:
checkpoint_path = './final_models_4/baseline_model1.ckpt'
model_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor='val_loss', save_best_only=True,
    save_weights_only=True, mode='min', save_freq='epoch')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/fit3/baseline_model1')
early_stopping = tf.keras.callbacks.EarlyStopping( monitor='val_loss', patience=60, min_delta=0.0001)

In [30]:
autoencoder1.fit(train_dataset_autoencoder1, epochs=300, validation_data = val_dataset_autoencoder1, callbacks=[model_callback, tensorboard_callback, early_stopping])

Epoch 1/300
1155/1155 [==============================] - 3s 2ms/step - loss: 18.1602 - mse: 18.1602 - val_loss: 16.4937 - val_mse: 16.4937
Epoch 2/300
1155/1155 [==============================] - 2s 2ms/step - loss: 14.2490 - mse: 14.2490 - val_loss: 12.6348 - val_mse: 12.6348
Epoch 3/300
1155/1155 [==============================] - 2s 2ms/step - loss: 11.0101 - mse: 11.0101 - val_loss: 9.9566 - val_mse: 9.9566
Epoch 4/300
1155/1155 [==============================] - 2s 2ms/step - loss: 8.5586 - mse: 8.5586 - val_loss: 7.4938 - val_mse: 7.4938
Epoch 5/300
1155/1155 [==============================] - 2s 2ms/step - loss: 6.5865 - mse: 6.5865 - val_loss: 6.0602 - val_mse: 6.0602
Epoch 6/300
1155/1155 [==============================] - 2s 2ms/step - loss: 5.4778 - mse: 5.4778 - val_loss: 5.0107 - val_mse: 5.0107
Epoch 7/300
1155/1155 [==============================] - 2s 2ms/step - loss: 4.4851 - mse: 4.4851 - val_loss: 4.0588 - val_mse: 4.0588
Epoch 8/300
1155/1155 [======================

In [31]:
tf.keras.backend.clear_session()

In [32]:
def generate_model_autoencoder2(input_len, name):
  input_encoder = Input(shape=(input_len,))  # input layer
  ip = Dropout(0.05)(input_encoder)
  
  encoder = Dense(6, activation='tanh')(ip)

  encoded = Dense(3, activation='tanh')(encoder)  # encoder ouput 

  decoder = Dense(6, activation='tanh')(encoded)

  decoded = Dense(input_len-1, activation='relu')(decoder) # decoder output 
  
  model = Model(input_encoder, decoded, name = name)

  return model

In [34]:
autoencoder2 = generate_model_autoencoder2(25,  "AutoencoderBasic-Cond")

In [35]:
autoencoder2 = compile_model(autoencoder2)

In [36]:
autoencoder2.summary()

Model: "AutoencoderBasic-Cond"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
dropout (Dropout)            (None, 25)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 156       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 21        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 24        
_________________________________________________________________
dense_3 (Dense)              (None, 24)                168       
Total params: 369
Trainable params: 369
Non-trainable params: 0
_______________________________________________

In [37]:
checkpoint_path = './final_models_4/baseline_model2.ckpt'
model_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor='val_loss', save_best_only=True,
    save_weights_only=True, mode='min', save_freq='epoch')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/fit3/baseline_model2')
early_stopping = tf.keras.callbacks.EarlyStopping( monitor='val_loss', patience=60, min_delta=0.0001)

In [38]:
train_dataset_autoencoder2 = tf.data.Dataset.from_tensor_slices((X_train_non_anom,Y_train_non_anom))

val_dataset_autoencoder2 = tf.data.Dataset.from_tensor_slices((X_val_non_anom, Y_val_non_anom))


BATCH_SIZE_TRAIN = 32
BATCH_SIZE_VAL = 16

train_dataset_autoencoder2 = train_dataset_autoencoder2.batch(BATCH_SIZE_TRAIN)
val_dataset_autoencoder2 = val_dataset_autoencoder2.batch(BATCH_SIZE_VAL)
print(train_dataset_autoencoder2)
print(val_dataset_autoencoder2)

<BatchDataset shapes: ((None, 25, 1), (None, 24, 1)), types: (tf.float64, tf.float32)>
<BatchDataset shapes: ((None, 25, 1), (None, 24, 1)), types: (tf.float64, tf.float32)>


In [39]:
autoencoder2.fit(train_dataset_autoencoder2, epochs=300, validation_data = val_dataset_autoencoder2, callbacks=[model_callback, tensorboard_callback, early_stopping])

Epoch 1/300
1155/1155 [==============================] - 2s 2ms/step - loss: 17.5396 - mse: 17.5396 - val_loss: 15.3530 - val_mse: 15.3530
Epoch 2/300
1155/1155 [==============================] - 2s 2ms/step - loss: 13.3266 - mse: 13.3266 - val_loss: 12.0745 - val_mse: 12.0745
Epoch 3/300
1155/1155 [==============================] - 2s 2ms/step - loss: 10.8054 - mse: 10.8054 - val_loss: 10.1214 - val_mse: 10.1214
Epoch 4/300
1155/1155 [==============================] - 2s 2ms/step - loss: 9.2619 - mse: 9.2619 - val_loss: 8.7256 - val_mse: 8.7256
Epoch 5/300
1155/1155 [==============================] - 2s 2ms/step - loss: 7.9116 - mse: 7.9116 - val_loss: 7.4930 - val_mse: 7.4930
Epoch 6/300
1155/1155 [==============================] - 2s 2ms/step - loss: 7.0750 - mse: 7.0750 - val_loss: 6.9133 - val_mse: 6.9133
Epoch 7/300
1155/1155 [==============================] - 2s 2ms/step - loss: 6.6859 - mse: 6.6859 - val_loss: 6.6251 - val_mse: 6.6251
Epoch 8/300
1155/1155 [====================

In [ ]:
#Add metric